# Split the watershed

In [ ]:
import matplotlib.pyplot as plt
import pathlib as pl
import numpy as np
import flopy
from flopy.mf6.utils import Mf6Splitter

from defaults import *

### Get the model directories for the base and the parallel simulations

In [ ]:
base_dir = get_base_dir()
parallel_dir = get_parallel_dir()

### Load base simulation data

In [ ]:
base_sim = flopy.mf6.MFSimulation.load(
    sim_ws=base_dir,
)

there is only one model in there:

In [ ]:
gwf = base_sim.get_model()
nrow = gwf.modelgrid.nrow
ncol = gwf.modelgrid.ncol

### Split into domains

Decide in how many domains you want to split. You might want to check how many cores you have. On Windows this will show in the Task manager:


<img src="./cpus.png" alt="drawing" width="700"/>

On Linux you can run the `lscpu` command and look for `Core(s) per socket:`.

In [ ]:
nr_domains = 2

Create the FloPy Model Splitter

In [ ]:
mfsplit = Mf6Splitter(base_sim)

Create the splitting array to assign groups of cells to a domain number. We either use Metis for partitioning or, apply a straightforward division into rectangular slices

In [ ]:
use_metis = False
if use_metis:
 split_array = mfsplit.optimize_splitting_mask(nparts=nr_domains)
else:
  split_array = np.zeros((nrow, ncol), dtype=int)
  for irow in range(nrow):
    for icol in range(ncol):      
      isub = np.floor(icol / (ncol / nr_domains))
      split_array[irow, icol] = isub

Plot the splitting array as a plan view and cut the inactive cells out

In [ ]:
pmv = flopy.plot.PlotMapView(model=gwf, layer=0)
pa = pmv.plot_array(split_array)
pmv.plot_inactive(color_noflow="white")
plt.ylabel("row")
plt.xlabel("column")
plt.colorbar(pa, shrink=0.6)

### Split the simulation into domains

In [ ]:
parallel_sim = mfsplit.split_model(split_array)

### HPC file
Get the models from the splitted simulation

In [ ]:
models = [parallel_sim.get_model(mname) for mname in parallel_sim.model_names]

Create the partition data for the HPC file

In [ ]:
partition_data = [[m.name, i] for i, m in enumerate(models)]
print(partition_data)

Add the package to the FloPy simulation 

In [ ]:
hpc = flopy.mf6.ModflowUtlhpc(parallel_sim, partitions=partition_data)

Now write the simulation to disk. Also write the lookup table from the splitter so we can recombine the data to represent a single domain further below

In [ ]:
parallel_sim.set_sim_path(parallel_dir)
parallel_sim.write_simulation()
mfsplit.save_node_mapping(parallel_dir / "mfsplit_node_mapping.json")

Plot the contours of the subdomains

In [ ]:
fig = plt.figure(figsize=(figwidth, 0.55*figheight))

for i,m in enumerate(parallel_sim.model_names):
  ax = fig.add_subplot(1,nr_domains,i+1)
  pmv = flopy.plot.PlotMapView(models[i])
  idomain = models[i].modelgrid.idomain
  pmv.plot_array(idomain[0], vmax=2)
  
  ax.set_xticklabels([])
  ax.set_yticklabels([])



Check if the model partitions are evenly split

In [ ]:
for m in models:
  idomain = m.modelgrid.idomain
  print(f"Nr. of active cells in {m.name}:", np.count_nonzero(idomain))
  